# Zadanie 9

In [1]:
import numpy as np
from math import ceil

In [2]:
def majority(neighbours):
    return int(sum(neighbours) > 1)


def GKL(initial_config, j, k, steps):
    diagram = [initial_config]
    n = len(initial_config)
    padded_config = np.pad(initial_config, (k, k), 'wrap')

    for _ in range(steps):
        neighbourhoods = []
        for idx in range(k, n + k):
            if padded_config[idx] == 0:
                neighbourhoods.append([padded_config[idx], padded_config[idx - j], padded_config[idx - k]])
            else:
                neighbourhoods.append([padded_config[idx], padded_config[idx + j], padded_config[idx + k]])

        result = [majority(neighbourhood) for neighbourhood in neighbourhoods]
        diagram.append(result)
        padded_config = np.pad(result, (k, k), 'wrap')

    return diagram


def DCP(diagram):
    initial_density = np.mean(diagram[0])  # Calculate initial density
    final_density = np.mean(diagram[-1])  # Calculate final density

    if initial_density < 0.5:
        return final_density == 0
    return final_density == 1


In [3]:
def convert_configuration(configuration: bytes, N: int):
    binary_str = ''.join(f'{byte:08b}' for byte in configuration)
    return np.array([int(bit) for bit in binary_str[:N]])


def configuration_reader(file_name: str, N: int, negate: bool):
    bytes_per_configuration = ceil(N / 8)
    configurations = []

    with open(file_name, 'rb') as file:
        while True:
            bytes_read = file.read(bytes_per_configuration)
            if not bytes_read:
                break

            cfg = convert_configuration(bytes_read, N)
            configurations.append(cfg)

            if negate:
                configurations.append(1 - cfg)

    return np.stack(configurations)

In [73]:
GKL([0, 1, 1, 0, 0, 1, 0, 0], 1, 3, 3)

[[0, 1, 1, 0, 0, 1, 0, 0],
 [0, 1, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

In [75]:
DCP(GKL([0, 1, 1, 0, 0, 1, 0, 0], 1, 3, 2))

False

In [4]:
configurations = configuration_reader("ALL_N21.bin", 21, True)

In [5]:
rules = [(4, 12), (3, 9), (2, 6), (5, 15), (1, 3), (1, 9), (1, 11), (2, 14), (2, 10), (1, 7), (1, 5)]

for rule in rules:
    results = []
    j = rule[0]
    k = rule[1]
    
    for config in configurations:
        results.append(DCP(GKL(config, j, k, 42)))
    
    print(f'Rule {rule}: {round(100 * sum(results) / len(results), 2)}% of correct solutions')
    

Rule (4, 12): 90.78% of correct solutions
Rule (3, 9): 17.67% of correct solutions
Rule (2, 6): 90.78% of correct solutions
Rule (5, 15): 90.78% of correct solutions
Rule (1, 3): 90.78% of correct solutions
Rule (1, 9): 67.6% of correct solutions
Rule (1, 11): 12.85% of correct solutions
Rule (2, 14): 36.01% of correct solutions
Rule (2, 10): 75.22% of correct solutions
Rule (1, 7): 36.01% of correct solutions
Rule (1, 5): 75.22% of correct solutions
